# South Australia Solar Supply Forecast

---

As South Australia progresses toward its 100% renewable energy target by 2030, having robust forecasting tools becomes ever more critical to maintain reliability as dependency on solar/wind increases. This project aims to forecast energy supply from solar photovoltaic (PV) in South Australia; a crucial step toward shaping sustainable energy strategies for the region.

### Data Source

Our data is collected with [OpenNEM API](https://opennem.org.au/) and [Open Meteo API](https://open-meteo.com/), made available under [CC BY 4.0 License](https://creativecommons.org/licenses/by/4.0/).

### Analysis Tool

We will be using Python 3 in this project.

---

## Navigating OpenNEM API
